<a href="https://colab.research.google.com/github/amun83/Financial-Data-Science/blob/master/03-02-Mid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2조  워드 임베딩을 이용한 관련주 분석 !


목적: 개별 한 종목을 조회해서 유사 종목군을 찾아 내기 위함.

# Source Import

훈련 데이터는 '날짜' [해당 날짜에 변동폭이 가장 큰 순서대로 정렬된 종목 데이터들]와 같은 형식이 반복되는 13년 치의 데이터로 구성되어져 있습니다.


In [0]:
from google.colab import files
uploaded = files.upload()

Saving traindata.txt to traindata.txt


# 1. Read Data from Text File



In [0]:
#traindata.txt는 13년치 금융 데이터에 대해서 day by day로 종목별 변동폭으로 정렬한 데이터입니다. hive를 사용하였습니다.
#13년치 데이터지만 데이터를 중간, 중간 삭제해서 절반인 2620일 -> 7년치 데이터입니다.

f = open("traindata.txt", 'r')
text=f.read()
# 전체 저장

print(text[:16158]) # 현재 텍스트 타입의 데이터라서 인덱스가 한 글자 단위라서 어림 짐작의 값을 인덱스로 3일자 데이터를 출력해봅니다.
print(len(text)) # 글자 수가 총 19009296개

2003-01-03	["014160","002710","031440","009415","004090","011000","005740","000950","002760","004565","004870","001140","004270","003465","006570","005810","009810","006345","001527","001465","003830","006110","008355","002005","000040","009070","002100","005420","009410","002960","002240","000545","001045","005620","010780","003300","005950","005820","001940","003200","002840","000145","016710","016380","021050","033780","000430","004990","001725","005305","004020","000300","000670","000970","001065","025540","011785","004135","001290","009680","004370","006980","017670","001040","016385","001740","036460","016450","015760","004545","000100","069620","006840","051900","000590","024090","039130","000070","011790","005390","009140","008250","006390","001745","051905","000850","005750","011280","003850","001790","007540","005320","008110","001720","002350","009320","003650","007210","010820","002355","023450","030200","000390","019175","016580","001460","003460","016090","001340","009450

2003-01-03에는 014160이라는 종목 코드를 가지는 종목이 가장 변동폭이 컸습니다.
다만, 이 데이터는 아직 전부 전처리 된 것은 아닙니다. 

#2. 데이터 전처리

Word2Vec의 입력으로 들어가려면 리스트의 형태로 바꾸어줘야 합니다.

Ex) [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

그래서 종목만 남겨두기 위해 일자 정보 및 기타 기호는 데이터에서 삭제할 예정입니다.

re 를 사용하여 데이터를 가공합니다.

- 날짜 제거
- [ , ], "," " 제거
- 종목코드만 리스트로 재작성

## 정규식처리 




In [0]:
import re
text=re.sub(r'....-..-..\t','',text)
# 날짜 제거
text[:10000]
text=re.sub(r']\n',']\n\n',text)
# 엔터 한번 더 추가
sentences=re.split(r'\n\n',text)
# 1차적으로 리스트로 나눔

print(sentences[0]) # 첫번째 데이터 출력

["014160","002710","031440","009415","004090","011000","005740","000950","002760","004565","004870","001140","004270","003465","006570","005810","009810","006345","001527","001465","003830","006110","008355","002005","000040","009070","002100","005420","009410","002960","002240","000545","001045","005620","010780","003300","005950","005820","001940","003200","002840","000145","016710","016380","021050","033780","000430","004990","001725","005305","004020","000300","000670","000970","001065","025540","011785","004135","001290","009680","004370","006980","017670","001040","016385","001740","036460","016450","015760","004545","000100","069620","006840","051900","000590","024090","039130","000070","011790","005390","009140","008250","006390","001745","051905","000850","005750","011280","003850","001790","007540","005320","008110","001720","002350","009320","003650","007210","010820","002355","023450","030200","000390","019175","016580","001460","003460","016090","001340","009450","004980",

첫번째 데이터를 출력하자 날짜가 제외된 2003년 1월 3일의 데이터가 나오는 것을 확인할 수 있습니다.

In [0]:
result = []
for sentence in sentences:
    sentence = re.sub(r'\[','',sentence)
    sentence = re.sub(r'\]','',sentence)
    sentence = re.sub(r'\"','',sentence)
    result.append(re.split(r',',sentence))
# 리스트의 리스트로 변경 성공

len(result)


2620

# 3. Training
## Word2Vec 적용






## ***Word2Vec(sentences=result, size=100, window=3, min_count=5, workers=4, sg=1)***

위 코드 의미는 다음과 같습니다. 

size = 100 : 포스태깅된 컨텐츠를 100차원의 벡터로 바꿔라. 

window = 3 : 주변 단어(window)는 앞뒤로 세개까지 본다.

min_count = 5 : 코퍼스 내 출현 빈도가 5번 미만인 단어는 분석에서 제외해라. 

workers = 4 : CPU는 쿼드코어 사용

sg = 1 : 분석방법론은 CBOW와 Skip-Gram 중 후자를 선택해라.

In [0]:
from gensim.models import Word2Vec # 이 데이터에 대해서 워드 임베딩을 해봅시다.
model = Word2Vec(sentences=result, size=100, window=3, min_count=5, workers=4, sg=1)


#4. 결과 분석

# most_similar()

gensim 패키지가 제공하는 기능 중에 ‘most_similar’라는 함수가 있습니다. 

두 벡터 사이의 코사인 유사도를 구해줍니다. 

그 값이 클 수록 비슷한 단어라는 뜻인데, 아래 코드는 현대차 "005380" 와 가장 비슷한(코사인 유사도가 큰) 종목을 나타냅니다.

In [0]:
case1=model.wv.most_similar("005380") #005380은 현대자동차
print(case1)

[('012330', 0.5735676288604736), ('002140', 0.5561268925666809), ('006400', 0.5484671592712402), ('000270', 0.5306755304336548), ('025000', 0.527518630027771), ('005420', 0.52409827709198), ('018880', 0.5234133005142212), ('013570', 0.5220799446105957), ('009150', 0.5156412124633789), ('005850', 0.5101193189620972)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


현대자동차(005380)에 대해서 유사도 순서대로 나온 것을 나열하면 다음과 같습니다.  

현대모비스012330

고려산업002140

삼성SDI006400

기아차000270

KPX케미칼025000

코스모화학005420

한온시스템018880

디와이013570

삼성전기009150

에스엘005850



In [0]:
case2=model.wv.most_similar("071050") # 한국금융지주
print(case2)

[('036530', 0.5685216784477234), ('001500', 0.565683126449585), ('037620', 0.5641739368438721), ('006360', 0.5633465051651001), ('008770', 0.5450915098190308), ('001290', 0.5384893417358398), ('004020', 0.5371870994567871), ('005680', 0.5364354848861694), ('005945', 0.5360994935035706), ('012450', 0.5283907651901245)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


한국금융지주(071050)에 대해서 유사도 순서대로 나온 것을 나열하면 다음과 같습니다.  

S&T홀딩스

현대차증권

미래에셋증권

GS건설

호텔신라

상상인증권

현대제철

삼영전자

NH투자증권우

한화에어로스페이스

In [0]:
case3=model.wv.most_similar("005940") # NH투자증권
print(case3)

[('003470', 0.6106569170951843), ('003535', 0.5636915564537048), ('000720', 0.5463590025901794), ('030610', 0.5424134731292725), ('016610', 0.5400291681289673), ('012450', 0.5369220972061157), ('010620', 0.5261048078536987), ('028050', 0.5251768827438354), ('017960', 0.5228593945503235), ('001515', 0.5222501754760742)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


NH투자증권(005940)에 대해서 유사도 순서대로 나온 것을 나열하면 다음과 같습니다.  

유안타증권

한화투자증권우

현대건설

교보증권

DB금융투자

한화에어로스페이스

현대미포조선

삼성엔지니어링

한국카본

In [0]:
case4=model.wv.most_similar("005930") # 삼성전자
print(case4)

[('012750', 0.5991263389587402), ('006660', 0.5931389331817627), ('003240', 0.5594978332519531), ('016710', 0.5532318353652954), ('025000', 0.5509283542633057), ('008490', 0.547899067401886), ('004690', 0.5414673089981079), ('001450', 0.5314470529556274), ('002240', 0.5304051041603088), ('000140', 0.5262224078178406)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [0]:
삼성전자에 대해서 유사도 순서대로 나온 것을 나열하면 다음과 같습니다.

에스원

삼성공조

태광산업

대성홀딩스

KPX케미칼

서흥

삼천리

현대해상

고려제강

하이트진로홀딩스